In [ ]:
import numpy as np

import pandas as pd
import geopandas as gpd

#import matplotlib.pyplot as plt
#import seaborn as sns

#import folium
import requests

import json

from random import randint

In [ ]:
from bokeh.io import output_file, show, output_notebook, export_png
from bokeh.models import ColumnDataSource, GeoJSONDataSource, LinearColorMapper, ColorBar, HoverTool, Select
from bokeh.plotting import figure
from bokeh.palettes import brewer
#from bokeh.sampledata.sample_geojson import geojson
from bokeh.layouts import widgetbox, row, column
from bokeh.io.doc import curdoc


from bokeh.tile_providers import CARTODBPOSITRON, get_provider

#output_notebook()

In [ ]:
data = requests.get('https://data.gov.au/geoserver/vic-suburb-locality-boundaries-psma-administrative-boundaries/wfs?request=GetFeature&typeName=ckan_af33dd8c_0534_4e18_9245_fc64440f742e&outputFormat=json', headers={"Content-Type":"application/json"})
geodata = data.json()

In [ ]:
with open('../data/raw/geodata.json', 'w', encoding='utf-8') as f:
    json.dump(geodata, f, indent=4)

In [ ]:
gdf = gpd.read_file('../data/raw/geodata.json')
gdf.set_crs('epsg:4283')
# map tile providers provide only a few projections, so casting to that specific projection
# WGS84 (EPSG:4326) or Web Mercator (EPSG:3857) are usually available 
# source: https://kodu.ut.ee/~kmoch/geopython2020/L6/interactive-map-bokeh.html
gdf['geometry'] = gdf['geometry'].to_crs(epsg=3857) 
gdf.head()


In [ ]:
gdf['base'] = 100
gdf['1bedroom_pred'] = np.random.randint(75, 125, gdf.shape[0])
gdf['2bedroom_pred'] = np.random.randint(100, 150, gdf.shape[0])

gdf.head()

In [ ]:
#gdf['geometry'] = gdf['geometry'].simplify(0.05, preserve_topology=True) # if we want to reduce precision of each polygon to make it quicker to display, 0.05 is the reducing coeficitient or whateve

In [ ]:
# Set bokeh to save file
output_file("../plots/test.html", title="Testing suburb plotting")

geo_source = GeoJSONDataSource(geojson=gdf.to_json())

# giving the basic view frame of the map
tile_provider = get_provider(CARTODBPOSITRON)
# range bounds supplied in web mercator coordinates
p = figure(x_range=(15600000, 16700000), y_range=(-5000000, -3900000),
           x_axis_type="mercator", y_axis_type="mercator",
           plot_height=600, plot_width=600)
p.add_tile(tile_provider)

# Define color palettes
palette = brewer['RdBu'][8]
# reverse order of colors so higher values have darker colors
palette = palette[::-1]

# Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
# TODO, use those as min and max from data we plot
color_mapper = LinearColorMapper(palette=palette, low=75, high=125)

color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8, width=500, height=20,
                     border_line_color=None, location=(0, 0), orientation='horizontal')
# major_label_overrides = tick_labels, could be used to set custom tickers, tick_labels is a dict value->display value


# Add patch renderer to figure.
suburbs = p.patches('xs', 'ys', source=geo_source,
                    # here comes the transform function
                    fill_color={'field': '2025_pred',
                                'transform': color_mapper},
                    line_color='gray',
                    line_width=0.25,
                    fill_alpha=0.2)

# add tooltips as needed for other prediction values
tooltips = [('Suburb', '@vic_loca_2'),
            ('2025 prediction', '@2025_pred')]
p.add_tools(HoverTool(renderers=[suburbs],
                      tooltips=tooltips))

show(p)  # this ; prevents vscode of showing output, output breaks things, todo to be fixef


In [ ]:
gdf.color = 'blue'
show(p)

In [ ]:
from bokeh.io import show
from bokeh.models import CustomJS, RadioGroup

LABELS = ["Option 1", "Option 2", "Option 3"]

radio_group = RadioGroup(labels=LABELS, active=0)
radio_group.js_on_click(CustomJS(code="""
    console.log('radio_group: active=' + this.active, this.toString())
"""))

show(radio_group)


In [ ]:
# might be needed
output_file("../plots/test2.html", title="Testing suburb plotting")

palette = brewer['RdBu'][8]
palette = palette[::-1]

# Define the callback function: update_plot


def update_plot(attr, old, new):
    # The input cr is the criteria selected from the select box
    appartment_type = select.value

    # Update the plot based on the changed inputs
    p = make_plot(appartment_type)

    # Update the layout, clear the old document and display the new document
    layout = column(p, widgetbox(select))
    curdoc().clear()
    curdoc().add_root(layout)

    # Create a plotting function


def make_plot(field_name):
    # colorbar, todo make actual minmax
    min_range = 75
    max_range = 150

    color_mapper = LinearColorMapper(
        palette=palette, low=min_range, high=max_range)

    # Create color bar.
    color_bar = ColorBar(color_mapper=color_mapper,
                         border_line_color=None, location=(0, 0))

    # Create figure object.
    apt_type = gdf[field_name]

    p = figure(x_range=(15600000, 16700000), y_range=(-5000000, -3900000),
               x_axis_type="mercator", y_axis_type="mercator",
               plot_height=600, plot_width=600,
               title=field_name+' prediction for 2025')

    # Add patch renderer to figure.
    geo_source = GeoJSONDataSource(geojson=gdf.to_json())
    suburbs = p.patches('xs', 'ys', source=geo_source,
                        fill_color={'field': field_name,
                                    'transform': color_mapper},
                        line_color='black', line_width=0.25, fill_alpha=0.5)

    # Specify color bar layout.
    p.add_layout(color_bar, 'right')

    # Add the hover tool to the graph
    # add tooltips as needed for other prediction values
    tooltips = [('Suburb', '@vic_loca_2'),
                ('1 bedroom 2025 prediction', '@1bedroom_pred'),
                ('2 bedroom 2025 prediction', '@2bedroom_pred')]
    hover = HoverTool(renderers=[suburbs], tooltips=tooltips)

    p.add_tools(hover)
    return p


# Call the plotting function
input_field = '1bedroom_pred'
p = make_plot(input_field)


# Make a selection object: select
# add here other column names you want to plot
select = Select(title='Select apt type:', value='1bedroom_pred',
                options=['1bedroom_pred', '2bedroom_pred'])
select.on_change('value', update_plot)

# Make a column layout of select and plot, and add it to the current document
# Display the current document
layout = column(p, select)
curdoc().add_root(layout)


# output_notebook()
show(p)
